In [1]:
"""
I completely dug this out of the main file not very intelligently, so at the moment this is likely messed up.
"""
%pip install -U "datasets>=2.14.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
# ──────────────────────────────────────────────────────────────
# Imports
# ──────────────────────────────────────────────────────────────
import json
import os
import re
import shutil
import shlex
import subprocess
from pathlib import Path
from glob import glob
from types import SimpleNamespace as ns

import polars as pl
import torch
from datasets import load_dataset, load_from_disk
from huggingface_hub import notebook_login
from google.colab import drive
from transformers import AutoTokenizer


# Setup

In [3]:
# ──────────────────────────────────────────────────────────────
# Global variables and namespace for the run
# ──────────────────────────────────────────────────────────────
# Datagen script settings
datagen_args = {
    'name': 'demo3s_cot',
    'length': 12,
    'dimension': 3,
    'train_samples': 10,
    'test_samples': 10,
    'cot_rate': 1,
    'no_filler_rate': 0,
    'corruption_rate': 1.33
}

# Run namespace
presets = {
    'cot': ns(
        runtype='cot',
        save_tag=None,
        filler_tok=None
    ),
    'direct': ns(
        runtype='direct',
        save_tag=None,
        filler_tok=None
    ),
    'filler_space-dot': ns(
        runtype='filler',
        save_tag='space-dot',
        filler_tok=' .'
    ),
    'filler_dot': ns(
        runtype='filler',
        save_tag='dot',
        filler_tok='.'
    ),
    'filler_space-neu': ns(
        runtype='filler',
        save_tag='space-neu',
        filler_tok=' neu'
    ),
    'filler_space-big': ns(
        runtype='filler',
        save_tag='space-big',
        filler_tok=' big'
    )
}
cfg = presets['cot']

# Core paths
BASE_DIR = "/content/drive/MyDrive/Colab_Files/repurposed_tokens"
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODEL_ID = "meta-llama/Llama-3.2-1B-Instruct"  # or 3-8B
MODEL_NAME = MODEL_ID.split('/')[-1]

RAW_TRAIN = os.path.join(DATA_DIR, 'base_train_w_cot.csv')
RAW_TEST = os.path.join(DATA_DIR, 'base_test_w_cot.csv')
RAW_ARGS = os.path.join(DATA_DIR, 'args_demo3s_cot.json')
print("RAW_TRAIN: ", RAW_TRAIN, "\n")

# Set data paths for each
for cfg in presets.values():
    tag = f"/{cfg.save_tag}" if cfg.save_tag else ""
    cfg.data_dir = f"{BASE_DIR}/data/{cfg.runtype}{tag}"
    print(cfg.runtype, cfg.data_dir)

    cfg.train_tok = f"{cfg.data_dir}/train_tokenized"
    cfg.test_tok = f"{cfg.data_dir}/test_tokenized"


RAW_TRAIN:  /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/base_train_w_cot.csv 

cot /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/cot
direct /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/direct
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/space-dot
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/dot
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/space-neu
filler /content/drive/MyDrive/Colab_Files/repurposed_tokens/data/filler/space-big


In [9]:
# ──────────────────────────────────────────────────────────────
# Github and Drive setup
# ──────────────────────────────────────────────────────────────
%cd /content

# Mount drive for data
drive.flush_and_unmount()
if os.path.exists('/content/drive') and not os.path.islink('/content/drive'):
    shutil.rmtree('/content/drive')
drive.mount('/content/drive', force_remount=True)

    # Now we can make the directories:
for cfg in presets.values():
    os.makedirs(cfg.data_dir, exist_ok=True)

# Filepaths
FILLER_DIR = "/content/fillerTokens"
ENTROPIX_DIR = "/content/entropix"

# Clone and setup repos
if not os.path.exists(FILLER_DIR):
    !git clone https://github.com/BreckEmert/fillerTokens.git
    !pip install -r {FILLER_DIR}/requirements.txt

if not os.path.exists(ENTROPIX_DIR):
    !git clone https://github.com/BreckEmert/entropix.git

# Set HF cache
os.environ["HF_DATASETS_CACHE"] = "/content/hf_ds_cache"


/content
Mounted at /content/drive


In [5]:
# ──────────────────────────────────────────────────────────────
# HF setup
# ──────────────────────────────────────────────────────────────
notebook_login()


# Math Dataset Generation Script

In [6]:
# ──────────────────────────────────────────────────────────────
# Generate raw math data with scripts.data_match3
# ──────────────────────────────────────────────────────────────
# This must be run because the script is a module:
%cd {FILLER_DIR}

# CSVs already exist
for raw_path in [RAW_TRAIN, RAW_TEST]:
    if os.path.exists(raw_path):
        continue

    # While you're figuring out filepathing this avoids accidental script runs
    # raise ValueError(f"Safety break.  Raw path: {raw_path}")

    # Run the data generation script
    cmd = (
        f"python -m scripts.data_match3 "
        f"--name {datagen_args['name']} "
        f"--length {datagen_args['length']} "
        f"--dimension {datagen_args['dimension']} "
        f"--train_samples {datagen_args['train_samples']} "
        f"--test_samples {datagen_args['test_samples']} "
        f"--cot_rate {datagen_args['cot_rate']} "
        f"--no_filler_rate {datagen_args['no_filler_rate']} "
        f"--corruption_rate {datagen_args['corruption_rate']} "
        f"--data_path {DATA_DIR}"
    )
    print("Generating ", raw_path, cmd)
    # subprocess.run(shlex.split(cmd))
    # If you need debug:
    result = subprocess.run(shlex.split(cmd), capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)

    # Args:
        # --length 12  # number of tuples per instance
        # --dimension 3  # number of integers per tuple
        # --cot_rate 0.5  # fraction to annotate with CoT traces
        # --no_filler_rate 0  # 0 so every non-CoT is punctuation-filler
        #  --corruption_rate 1.33  # Generated nstances have avg 1.33 digits randomly replaced (False are modified True examples and made False by replacing digits)
    # Generates:
        # args_demo3s_YYYY-MM-DD.json  # metadata
        # demo3s_trainset_YYYY-MM-DD.csv
        # demo3s_testset_YYYY-MM-DD.csv

    # Find the generated file
    new_train = glob(f"{DATA_DIR}/demo3s_cot_trainset_*.csv")[0]
    new_test = glob(f"{DATA_DIR}/demo3s_cot_testset_*.csv")[0]
    new_arg = glob(f"{DATA_DIR}/args_demo3s_cot_*.json")[0]

    # Remove the date from their names
    Path(new_train).rename(RAW_TRAIN)
    Path(new_test).rename(RAW_TEST)
    Path(new_arg).rename(new_arg[:-16] + ".json")

print(f"Train csv: {pl.scan_csv(RAW_TRAIN).describe()}\n")
print(f"Test csv: {pl.scan_csv(RAW_TEST).describe()}\n")
print(f"Config json: {json.load(open(RAW_ARGS))}")

!ls {cfg.data_dir}


/content/fillerTokens
Train csv: shape: (9, 2)
┌────────────┬─────────────────────────────────┐
│ statistic  ┆  981 545 417 331 968 408 842 6… │
│ ---        ┆ ---                             │
│ str        ┆ str                             │
╞════════════╪═════════════════════════════════╡
│ count      ┆ 9                               │
│ null_count ┆ 0                               │
│ mean       ┆ null                            │
│ std        ┆ null                            │
│ min        ┆  040 124 964 515 080 353 229 0… │
│ 25%        ┆ null                            │
│ 50%        ┆ null                            │
│ 75%        ┆ null                            │
│ max        ┆  933 538 904 464 222 736 537 4… │
└────────────┴─────────────────────────────────┘

Test csv: shape: (9, 2)
┌────────────┬─────────────────────────────────┐
│ statistic  ┆  957 244 213 925 238 863 264 8… │
│ ---        ┆ ---                             │
│ str        ┆ str                            

# Make Tokenized Datasets

In [7]:
# ──────────────────────────────────────────────────────────────
# Generate Tokenized datasets
# ──────────────────────────────────────────────────────────────
# Instantiate tokenizer
tok = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)
tok.pad_token = tok.eos_token

# Establish tokens we need to reference
tok.cot_begin_id = tok(" P", add_special_tokens=False)["input_ids"]
tok.cot_end_id = tok(" A", add_special_tokens=False)["input_ids"]
tok.true_id = tok(" True", add_special_tokens=False)["input_ids"]
tok.false_id = tok(" False", add_special_tokens=False)["input_ids"]

assert all(len(id) == 1 for id in [
    tok.cot_begin_id, tok.cot_end_id, tok.true_id, tok.false_id
]), "One of cot_id or truthy_id is not one id"

# Change from lists to ints
tok.cot_begin_id = tok.cot_begin_id[0]
tok.cot_end_id = tok.cot_end_id[0]
tok.true_id = tok.true_id[0]
tok.false_id = tok.false_id[0]

# Set each preset's filler token
for cfg in presets.values():
    if cfg.filler_tok:
        cfg.filler_id = tok(
            cfg.filler_tok, add_special_tokens=False
        )["input_ids"]
        assert len(cfg.filler_id) == 1, f"tok({cfg.filler_tok}) is not one id."
        cfg.filler_id = cfg.filler_id[0]
    else:
        cfg.filler_id = None

# Function to tokenize and mask a batch
def tokenize_and_mask(batch):
    # We want the labels to be only on the answer for all runs
    # We can do some special tokens stuff but I'm just going to write this for now - and fix that later.  Because, I think we need to get prompt template stuff anyway?  Should we even have an instruct model - probably not?  And the only thing a base model needs is bos and eos right?  Chat templates are such a nightmare for so many reasons.
    ids_batch = tok(batch['prompt'], add_special_tokens=False)["input_ids"]
    labels = [[-100] * (len(ids)-1) + [ids[-1]] for ids in ids_batch]
    return {"input_ids": ids_batch, 'labels': labels}

# Read the raw csv (raw cot script generation) as a HF dataset
raw_ds = load_dataset(
    "csv",
    data_files={"train": RAW_TRAIN, "test": RAW_TEST},
    column_names=["prompt"]
)

# Tokenize the raw dataset
def encode(batch):
    enc = tok(batch["prompt"], add_special_tokens=False)
    ids = enc["input_ids"]
    return {
        "input_ids": ids,
        # only the final token (answer / eos) is supervised
        "labels": [[-100] * (len(x) - 1) + [x[-1]] for x in ids],
    }
tok_ds = raw_ds.map(encode, batched=True, remove_columns=["prompt"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [23]:
# ──────────────────────────────────────────────────────────────
# Generate run-specific datasets
# ──────────────────────────────────────────────────────────────
# Function to replace the filler of runtype='filler' datasets
l = datagen_args['length']*2 + 1  # Spaces so *2, 1 for trailing space
def replace_filler(ids: list[int], filler_id: int) -> list:
    if filler_id:
        # Make the new filler
        n_tokens = ids.index(tok.cot_end_id) - l
        new_filler = [filler_id] * n_tokens
    else:
        new_filler = []

    # Return the original q/a with the new filler
    return ids[:l] + new_filler + [ids[-1]]

# Loop through all presets
for name, cfg in presets.items():
    # Don't generate if it already exists
    if os.path.exists(cfg.train_tok) and os.path.exists(cfg.test_tok):
        print(f"{name} already exists.")
        # continue

    print(f"Running {name}")
    if cfg.runtype == 'cot':
        # Base csv is just the cot generation
        final_ds = tok_ds
    else:
        # Need to replace the filler
        final_ds = tok_ds.map(
            lambda batch, fid=cfg.filler_id: {
                # rewrite the filler tokens
                "input_ids": (new_ids := [
                    replace_filler(ids, fid) for ids in batch["input_ids"]
                ]),
                # rebuild the labels to match the rewritten ids
                "labels": [[-100]*(len(ids)-1) + [ids[-1]] for ids in new_ids],
            }, batched=True
        )

    # Save to disk
    final_ds['train'].save_to_disk(cfg.train_tok)
    final_ds['test'].save_to_disk(cfg.test_tok)


cot already exists.
Running cot


Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

direct already exists.
Running direct


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

...

filler_space-big already exists.
Running filler_space-big


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10 [00:00<?, ? examples/s]

# Debug Cells

In [27]:
# AI GENERATED DEBUG CELL
# ──────────────────────────────────────────────────────────────
# Super-verbose debug / inspection cell
# ──────────────────────────────────────────────────────────────
from datasets import load_from_disk
from itertools import islice

N_EXAMPLES = 1        # rows per split to inspect

def inspect(ds, split_name, cfg_name):
    for ex_idx in range(min(N_EXAMPLES, len(ds))):
        ids = ds[ex_idx]["input_ids"]
        lbl = ds[ex_idx]["labels"]
        toks = tok.convert_ids_to_tokens(ids, skip_special_tokens=False)

        print(f"\n── {cfg_name} | {split_name} | example {ex_idx} ──")
        print("pos | token                    | id         | label_id")
        print("----+---------------------------+------------+----------")
        for pos, (t, i, l) in enumerate(zip(toks, ids, lbl)):
            print(f"{pos:3d} | {t:<25.25} | {i:<10d} | {l}")
            if l != -100 and l != i:
                raise ValueError(
                    f"Label/input mismatch at pos {pos}: input={i}, label={l}"
                )

for name, cfg in presets.items():
    if name != 'cot':
        continue

    print(f"\n================= {name.upper()} =================")
    train_ds = load_from_disk(cfg.train_tok)
    test_ds = load_from_disk(cfg.test_tok)

    inspect(train_ds, "train", name)
    inspect(test_ds, "test", name)

print("\n✓ Finished inspecting all presets.")



================= COT =================

── cot | train | example 0 ──
pos | token                    | id         | label_id
----+---------------------------+------------+----------
  0 | Ġ                         | 220        | -100
  1 | 981                       | 25643      | -100
  2 | Ġ                         | 220        | -100
  3 | 545                       | 20749      | -100
  4 | Ġ                         | 220        | -100
  5 | 417                       | 19561      | -100
  6 | Ġ                         | 220        | -100
  7 | 331                       | 16707      | -100
  8 | Ġ                         | 220        | -100
  9 | 968                       | 23386      | -100
 10 | Ġ                         | 220        | -100
 11 | 408                       | 18058      | -100
 12 | Ġ                         | 220        | -100
 13 | 842                       | 25377      | -100
 14 | Ġ                         | 220        | -100
 15 | 616                       | 21

In [25]:
tok(' P', add_special_tokens=False)['input_ids']

[393]